<td>
   <a target="_blank" href="https://labelbox.com" ><img src="https://labelbox.com/blog/content/images/2021/02/logo-v4.svg" width=256/></a>
</td>

<td>
<a href="https://colab.research.google.com/github/Labelbox/labelbox-python/blob/develop/examples/annotation_import/subclasses.ipynb" target="_blank"><img
src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>
</td>

<td>
<a href="https://colab.research.google.com/github/Labelbox/labelbox-python/blob/develop/examples/annotation_import/subclasses.ipynb" target="_blank"><img
src="https://img.shields.io/badge/GitHub-100000?logo=github&logoColor=white" alt="GitHub"></a>
</td>

# Image Annotation Import with Subclasses
* This notebook will provide examples of each supported annotation type for image assets. It will cover the following:
    * Model-assisted labeling - used to provide pre-annotated data for your labelers. This will enable a reduction in the total amount of time to properly label your assets. Model-assisted labeling does not submit the labels automatically, and will need to be reviewed by a labeler for submission.
    * Label Import - used to provide ground truth labels. These can in turn be used and compared against prediction labels, or used as benchmarks to see how your labelers are doing.
    * Right now, our *Annotation Types* schemas don't support nested classifications. So, we'll be using NDJSON syntax in this tutorial.


* For information on what types of annotations are supported per data type, refer to this documentation:
    * https://docs.labelbox.com/docs/model-assisted-labeling#option-1-import-via-python-annotation-types-recommended
* For more information on the syntax of annotations with nested classifications, view our guide here:
    * https://docs.labelbox.com/recipes/import-annotations

* Notes:
    * If you are importing more than 1,000 mask annotations at a time, consider submitting separate jobs, as they can take longer than other annotation types to import.
    * Wait until the import job is complete before opening the Editor to make sure all annotations are imported properly.

# Installs

In [ ]:
!pip install -q 'labelbox[data]'

# Imports

In [2]:
from labelbox.schema.ontology import OntologyBuilder, Tool, Classification, Option
from labelbox import Client, LabelingFrontend, LabelImport, MALPredictionImport
from labelbox.data.annotation_types import (
    Label, ImageData, ObjectAnnotation, MaskData,
    Rectangle, Point, Line, Mask, Polygon,
    Radio, Checklist, Text,
    ClassificationAnnotation, ClassificationAnswer
)
from labelbox.data.serialization import NDJsonConverter
import uuid
import json
import numpy as np
import copy

# API Key and Client
Provide a valid api key below in order to properly connect to the Labelbox Client.

In [38]:
# Add your api key
API_KEY = None
client = Client(api_key=API_KEY)

The Syntax between model-assisted labeling and label import are the same - have one of these set to `True` and the other set to `False`. The only difference here is that:
- Model-assisted labeling uploads annotations to-be-labeled. As in, they aren't submitted
- Label Import uploads annotations as submitted labels. That way, they can be sent straight to the review step (if enabled) or used as a ground truth label

In [39]:
# Have one value here set to True, the other to False
model_assisted_labeling = False
label_import = True

---- 
### Steps
1. Make sure project is setup
2. Collect annotations
3. Upload

### Project setup

We will be creating two projects, one for model-assisted labeling, and one for label imports

In [66]:
ontology_builder = OntologyBuilder(
    tools=[
        Tool(tool=Tool.Type.BBOX, name="box",
             classifications=[
                 Classification(
                     class_type=Classification.Type.CHECKLIST, 
                     instructions="tool_nested_checklist", 
                     options=[Option(value="first_tool_nested_checklist_answer"),
                              Option(value="second_tool_nested_checklist_answer"),
                              Option(value="third_tool_nested_checklist_answer")]),
                 Classification(
                     class_type=Classification.Type.RADIO, 
                     instructions="tool_nested_radio", 
                     options=[Option(value="first_tool_nested_radio_answer"),
                              Option(value="second_tool_nested_radio_answer")],
                              )])
        ],
    classifications=[
        Classification(
            class_type=Classification.Type.RADIO, 
            instructions="radio", 
            options=[Option(value="first_radio_answer",
                            options=[
                                Classification(
                                  class_type=Classification.Type.CHECKLIST,
                                  instructions="nested_checklist",
                                  options=[
                                      Option(value="first_nested_checklist_answer"),
                                      Option(value="second_nested_checklist_answer"),
                                      Option(value="third_nested_checklist_answer")
                                  ])]),
                    Option(value="second_radio_answer"),
                    Option(value="third_radio_answer")])
    ]
)

In [67]:
subclasses_project = client.create_project(name="subclass_annotation_import_project")

dataset = client.create_dataset(name="subclass_annotation_import_demo_dataset")
test_img_url = "https://raw.githubusercontent.com/Labelbox/labelbox-python/develop/examples/assets/2560px-Kitano_Street_Kobe01s5s4110.jpg"
data_row = dataset.create_data_row(row_data=test_img_url)
editor = next(client.get_labeling_frontends(where=LabelingFrontend.name == "Editor"))

subclasses_project.setup(editor, ontology_builder.asdict())
subclasses_project.datasets.connect(dataset)

Creates a dictionary where the key is the name of the tool, classification or option and the value is the ontology's featureSchemaId

In [68]:
def map_features(ontology_normalized):
  """ Creates a dictionary where keys = tool/classification/option name : values = featureSchemaId
  Args:
    ontology_normalized   :   Queried from a project using project.ontology().normalized
  Returns:
    Dictionary with the specified keys and values
  """
  feature_map = {}
  tools = ontology_normalized["tools"]
  classifications = ontology_normalized["classifications"]
  if tools:
    feature_map, next_layer = layer_iterator(feature_map, tools)
  if classifications:
    feature_map, next_layer = layer_iterator(feature_map, classifications)
  return feature_map

def layer_iterator(feature_map, node_layer):
  for node in node_layer:
    try:
      node_name = node["name"]
      next_layer = node["classifications"]
    except:
      try:
        node_name = node["instructions"]
        next_layer = node["options"]
      except:
        node_name = node["label"]
        try:
          next_layer = node["options"]
        except:
          next_layer = []
    feature_map.update({node_name : node['featureSchemaId']})
    feature_map = copy.deepcopy(feature_map)
    if next_layer:
      feature_map, next_layer = layer_iterator(feature_map, next_layer)
  return feature_map, next_layer

In [69]:
name_schemaId_map = map_features(subclasses_project.ontology().normalized)

In [70]:
name_schemaId_map

{'box': 'cl1dnu41o0ipt0z40bjoletaj',
 'first_nested_checklist_answer': 'cl1dnu41p0iqc0z40cgov1soq',
 'first_radio_answer': 'cl1dnu41p0iqa0z40gh8p9y7t',
 'first_tool_nested_checklist_answer': 'cl1dnu41o0ipv0z404sth992b',
 'first_tool_nested_radio_answer': 'cl1dnu41o0iq30z4075ae16n5',
 'nested_checklist': 'cl1dnu41p0iqb0z40gui5fqee',
 'radio': 'cl1dnu41p0iq90z4012iu9vy4',
 'second_nested_checklist_answer': 'cl1dnu41p0iqe0z40fw6nf9vj',
 'second_radio_answer': 'cl1dnu41p0iqk0z40c4gz45wl',
 'second_tool_nested_checklist_answer': 'cl1dnu41o0ipx0z40fuvk0tb6',
 'second_tool_nested_radio_answer': 'cl1dnu41o0iq50z4052nbe6jr',
 'third_nested_checklist_answer': 'cl1dnu41p0iqg0z408uws98rt',
 'third_radio_answer': 'cl1dnu41p0iqm0z401k4p324k',
 'third_tool_nested_checklist_answer': 'cl1dnu41o0ipz0z406j7qhdv2',
 'tool_nested_checklist': 'cl1dnu41o0ipu0z40awi6ap1f',
 'tool_nested_radio': 'cl1dnu41o0iq20z403oxvgw7t'}

### Create Label using NDJSON Syntax
* For subclasses on an annotation, use te below syntax

In [71]:
tool_subclass_list = [
  {
    'schemaId' : name_schemaId_map['tool_nested_radio'],
    'answer' : { 'schemaId' : name_schemaId_map['second_tool_nested_radio_answer']}
  },
  {
    'schemaId' : name_schemaId_map['tool_nested_checklist'],
    'answers' : [{ 'schemaId' : name_schemaId_map['second_tool_nested_checklist_answer']}, { 'schemaId' : name_schemaId_map['third_tool_nested_checklist_answer']}]
  }
]

tool_subclass_list

[{'answer': {'schemaId': 'cl1dnu41o0iq50z4052nbe6jr'},
  'schemaId': 'cl1dnu41o0iq20z403oxvgw7t'},
 {'answers': [{'schemaId': 'cl1dnu41o0ipx0z40fuvk0tb6'},
   {'schemaId': 'cl1dnu41o0ipz0z406j7qhdv2'}],
  'schemaId': 'cl1dnu41o0ipu0z40awi6ap1f'}]

In [72]:
classification_subclass_list = [
  {
    'schemaId' : name_schemaId_map['nested_checklist'],
    'answers' : [{ 'schemaId' : name_schemaId_map['first_nested_checklist_answer']}]
  }
]

classification_subclass_list

[{'answers': [{'schemaId': 'cl1dnu41p0iqc0z40cgov1soq'}],
  'schemaId': 'cl1dnu41p0iqb0z40gui5fqee'}]

### Create an Annotation NDJSON
* This will result in one NDJSON per annotation
* One annotation can have a list of classifications. 
* The `answer` field always need a `schemaId`, but can also have a `classifications` field if your annotation has more layers

In [73]:
tool_annotation = {
    'uuid' : str(uuid.uuid4()),
    'schemaId' : name_schemaId_map['box'],
    'dataRow' : {'id' : data_row.uid},
    'bbox' : {
        'top' : 50,
        'left' : 50,
        'height' : 200,
        'width' : 200
    },
    'classifications' : tool_subclass_list
}
dict(tool_annotation)

{'bbox': {'height': 200, 'left': 50, 'top': 50, 'width': 200},
 'classifications': [{'answer': {'schemaId': 'cl1dnu41o0iq50z4052nbe6jr'},
   'schemaId': 'cl1dnu41o0iq20z403oxvgw7t'},
  {'answers': [{'schemaId': 'cl1dnu41o0ipx0z40fuvk0tb6'},
    {'schemaId': 'cl1dnu41o0ipz0z406j7qhdv2'}],
   'schemaId': 'cl1dnu41o0ipu0z40awi6ap1f'}],
 'dataRow': {'id': 'cl1dnu3cc0iec0z2w1hbd85mr'},
 'schemaId': 'cl1dnu41o0ipt0z40bjoletaj',
 'uuid': '4e043a88-e7b5-45b9-a04e-0e63a273bfba'}

In [74]:
classification_annotation = {
    'uuid' : str(uuid.uuid4()),
    'schemaId' : name_schemaId_map['radio'],
    'answers' : {
        'schemaId' : name_schemaId_map['first_radio_answer'],
    },
    'classifications' : classification_subclass_list,
    'dataRow' : {'id' : data_row.uid}
}
dict(classification_annotation)

{'answers': {'schemaId': 'cl1dnu41p0iqa0z40gh8p9y7t'},
 'classifications': [{'answers': [{'schemaId': 'cl1dnu41p0iqc0z40cgov1soq'}],
   'schemaId': 'cl1dnu41p0iqb0z40gui5fqee'}],
 'dataRow': {'id': 'cl1dnu3cc0iec0z2w1hbd85mr'},
 'schemaId': 'cl1dnu41p0iq90z4012iu9vy4',
 'uuid': '864dc092-4d3a-4e9c-97f4-0143b67210f3'}

In [75]:
ndjson_annotations = [tool_annotation, classification_annotation]

### Model Assisted Labeling 

If `model_assisted_labeling` is `True`, this section will execute, uploading annotations as pre-labels. 

In [76]:
if model_assisted_labeling:
  upload_job = MALPredictionImport.create_from_objects(
      client = client, 
      project_id = subclasses_project.uid, 
      name="upload_label_import_job", 
      predictions=ndjson_annotations)

In [77]:
# Errors will appear for each annotation that failed.
# Empty list means that there were no errors
# This will provide information only after the upload_job is complete, so we do not need to worry about having to rerun
if model_assisted_labeling:
  print("Errors:", upload_job.errors)

### Label Import

If `label_import` is `True`, this section will execute, uploading annotations as submitted labels. Label import is very similar to model-assisted labeling except that it will submit annotations uploaded.

In [78]:
if label_import:
  upload_job = LabelImport.create_from_objects(
      client = client, 
      project_id = subclasses_project.uid, 
      name="upload_label_import_job", 
      labels=ndjson_annotations)

In [79]:
if label_import:
  print("Errors:", upload_job.errors)

Errors: []
